In [ ]:

from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content/drive/My Drive/ISODS/SCUT-FBP5500_v2"
%cd {path}
!ls

Mounted at /content/drive
/content/drive/My Drive/ISODS/SCUT-FBP5500_v2
 All_Ratings.xlsx  'facial landmark'   Images_Sources.xlsx   train_test_files
 epoch_ViT.ipynb    Images	       README.txt	     VIT_pretrain_model


In [ ]:
!ls

 All_Ratings.xlsx  'facial landmark'   Images_Sources.xlsx   train_test_files
 epoch_ViT.ipynb    Images	       README.txt	     VIT_pretrain_model


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.4 MB/s eta 0:00:00


In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 11.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image

from transformers import AutoImageProcessor, ViTForImageClassification
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchmetrics import MeanAbsoluteError

# Define hyperparameters
num_classes = 1  # regression task
image_size = 224
batch_size = 32
num_epochs = 15
learning_rate = 0.0001
# Define hyperparameters
# learning_rate = 0.001
weight_decay = 0.01  # Set the weight decay value
betas = (0.9, 0.999)  # Beta parameters for Adam optimizer


# Load pre-trained model and feature extractor
model_name = 'google/vit-base-patch16-224'
model_face = "jayanta/google-vit-base-patch16-224-face"
feature_extractor = AutoImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_face, num_labels=num_classes, ignore_mismatched_sizes=True)



Some weights of ViTForImageClassification were not initialized from the model checkpoint at jayanta/google-vit-base-patch16-224-face and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([7, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([7]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define custom dataset and dataloader
class SCUT5500Dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file, index_col = 0)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        img_path = f"{self.root_dir}/{img_name}"
        image = Image.open(img_path).convert('RGB')
        label = self.data.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()

In [ ]:
dataset = SCUT5500Dataset(csv_file='Images/csv/train.csv', root_dir='Images', transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor()
]))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.AdamW(model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)

torch.manual_seed(16)

model.to(device)
epoch_loss_min = 9999999
# Fine-tune model on SCUT-5500 dataset
for epoch in range(num_epochs):
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            # Extract features from image and pass through model's last layer
            features = feature_extractor(inputs, return_tensors='pt')
            features = features.to(device)
            outputs = model(**features) 
            loss = criterion(outputs.logits, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataset)
    if epoch_loss_min > epoch_loss:
      epoch_loss_min = epoch_loss
      model.save_pretrained("VIT_pretrain_model")
    print(f"Epoch {epoch+1}/{num_epochs} loss: {epoch_loss:.4f}")

0
Epoch 1/15 loss: 0.3397
1
Epoch 2/15 loss: 0.0784
2
Epoch 3/15 loss: 0.0441
3
Epoch 4/15 loss: 0.0233
4
Epoch 5/15 loss: 0.0143
5
Epoch 6/15 loss: 0.0141
6
Epoch 7/15 loss: 0.0100
7
Epoch 8/15 loss: 0.0081
8
Epoch 9/15 loss: 0.0057
9
Epoch 10/15 loss: 0.0050
10
Epoch 11/15 loss: 0.0053
11
Epoch 12/15 loss: 0.0067
12
Epoch 13/15 loss: 0.0085
13


KeyboardInterrupt: ignored

In [ ]:
testset = SCUT5500Dataset(csv_file='Images/csv/test.csv', root_dir='Images', transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor()
]))
test_dataloader = DataLoader(testset, batch_size=batch_size, shuffle=True)
model = ViTForImageClassification.from_pretrained("VIT_pretrain_model")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [ ]:
import numpy as np
import torchmetrics
import numpy as np
from torchmetrics import MeanAbsoluteError, MeanSquaredError, PearsonCorrCoef

mean_absolute_error = MeanAbsoluteError().to(device)
mean_squared_error = MeanSquaredError().to(device)
# pc = PearsonCorrCoef().to(device)
pc_list = []
sum_mae = 0.0
sum_mse = 0.
for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        # Extract features from image and pass through model's last layer
        features = feature_extractor(inputs, return_tensors='pt')
        features = features.to(device)
        outputs = model(**features) 
        temp_mae = mean_absolute_error(outputs.logits, labels.unsqueeze(1).float())
        temp_mse = mean_squared_error(outputs.logits, labels.unsqueeze(1).float())
        # print(outputs.logits.numpy(), labels.unsqueeze(1).float().numpy())
        temp_pc = np.corrcoef(outputs.logits.cpu(), labels.unsqueeze(1).float().cpu(), rowvar = False)[0][1]
        
        pc_list.append(temp_pc)
      
    sum_mae += temp_mae * inputs.size(0)
    sum_mse += temp_mse * inputs.size(0)


In [ ]:
mae = sum_mae/len(testset)
mse = sum_mse/len(testset)
rmse = np.sqrt(mse.cpu())
pc = np.mean(pc_list)
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"PC: {pc}")


MAE: 0.21979981660842896
MSE: 0.083194300532341
RMSE: 0.2884342074394226
PC: 0.9052464018474873
